# GAN IMPLEMENTATION FROM COMMUNITY INQ28 MODELS
- Code Author: Jack MacCormick
- Instagram: argelpaints: https://www.instagram.com/argelpaints/
- Done in partnership with 28MAG: https://28-mag.com/

**GAN Training and Development Code:**

This project is the use of GAN style Machine Learning algorithms to generate new images, based off the works of the Warhammer INQ28 community. These images are the dreamings of an abhorent AI system, who's task is to understand "what is INQ28" it knows not the world, it's surroundings, only those offered to it, and the endless void

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# import necessary packages
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pylab
import h5py

from PIL import Image
import pandas as pd
import tensorflow as tf
from tensorflow import keras

import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import random
from numpy.random import randint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

from tqdm import tqdm 
from IPython import display 

In [3]:
learningRate = 0.00015
n_epochs = 50 # number of "generations"
n_batch = 20 # no. of images in each batch

In [4]:
"""
define the standalone discriminator model
key value is the learning rate lr

"""

def define_discriminator(in_shape=(200,200,1)):
	# weight initialization
	init = 'uniform'
	# define model
	model = Sequential()
	# downsample to 100x100
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 50x50
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(learning_rate=0.0001, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [5]:
def define_generator(latent_dim):
	# weight initialization
	init = 'uniform'
	# define model
	model = Sequential()
	# foundation for 50x50 image
	n_nodes = 256 * 50 * 50
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((50, 50, 256)))
	# upsample to 100x100
	model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 200x200
	model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# output 200x200x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model

In [28]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False # making this True means it never learns, both accs == 100% all the time, gens a black square every time
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	opt = Adam(learning_rate=learningRate, beta_1=0.7) # lr=00015 seems to be the ideal value for training
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [7]:
# default values of standardised images
preferred_width = 200
preferred_height = 200

In [8]:
pathSource = '/content/drive/MyDrive/BnWDrive'
dir_list = os.listdir(pathSource)

In [9]:
images = []
# for image in pathSource, convert them to BnW, save that to pathDest, then add standardised version to images[]
for image in dir_list:
    img = Image.open(pathSource+"/"+image)
    imgArray = np.array(img) # int array of pixels
    imgArray = imgArray.astype(float)/255 # standardise values to be 0:255
    images.append(imgArray)


In [10]:
# setting a param for proportion of data to keep for testing
testingSize = int(len(dir_list)/5)

X_testing = np.array(images[0:testingSize])
X_training = np.array(images[testingSize:len(images)])

"""
# this is all diagnostic checks to ensure data is passed correctly to training and testing splits

# code:
print(np.shape(X_testing)) 
print(X_testing[1]) # checking it's correctly put something into the testing variable
print(np.shape(X_training))
# end code
"""

()

()

In [11]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y

In [12]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.uniform(low=0.0, high=1.0, size=(latent_dim, n_samples))
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [13]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = abs(generate_latent_points(latent_dim, n_samples))
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n_samples, 1))
	return X, y

In [ ]:
"""
trains the generator and discriminator parts of the GAN
saves the net model to a .h5 file
"""


'''
# done in train+draw function

def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs, n_batch):
	# calculate the number of batches per epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the total iterations based on batch and epoch
	n_steps = bat_per_epo * n_epochs
	# calculate the number of samples in half a batch
	half_batch = int(n_batch / 2)
	# prepare lists for storing stats each iteration
	d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
		# generate 'fake' examples
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d/%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
			(i+1, n_steps, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
		# record history
		d1_hist.append(d_loss1)
		d2_hist.append(d_loss2)
		g_hist.append(g_loss)
		a1_hist.append(d_acc1)
		a2_hist.append(d_acc2)
		if (i%10==0): # save frequency, this should really be a smart value
			g_model.save('WARgenerator.h5')		
	print("finished training")
	# this is the generator that can be called externally
	g_model.save('WARgenerator.h5') 

  '''
  
	"""
	Save is also embed this within the loop every so many itterations
	so something is saved/retained if user needs to abort while long training processes are occuring or if something fails, if the kernel or local machine crashes or if local machine looses power
	"""

In [14]:
"""
set up the data into something the GAN can use in training
"""

X_train=np.reshape(X_training,(X_training.shape[0], X_training.shape[1],X_training.shape[2],1))

In [29]:
"""
set up for training and training call
"""

# training params
latent_dim = 50 # size of the latent space
# training above 50 generations, results stabilise, and no change is noticable

# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = X_train


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
"""
'''
this can take some time depending on number of epochs
recommend running on a GPU
'''

# train model
train(generator, discriminator, gan_model, dataset, latent_dim, n_epochs, n_batch)
"""

In [ ]:
"""
getting the model to draw something,
this can be done seperately, if put in another file
"""

''' 
currently doing in func

model = load_model('WARgenerator.h5', compile = False)
# generate images
latent_points = generate_latent_points(latent_dim, 1)
 # has quite the think about it
predictionInstance = model.predict(latent_points)
print("done thinking")
'''

In [ ]:
"""
Training of existing model file:
May not always want to run this cell
"""

'''
n_epochs = 10
train(model, discriminator, gan_model, dataset, latent_dim, n_epochs, n_batch)
'''

In [ ]:
"""
getting the prediction directly from generator
functionally the same as prediction instance
"""

''' 
currently doing in func

xout = generate_fake_samples(generator, latent_dim, n_samples = 1)[0]
# predictionInstance = model.predict(latent_points)
# convert floats from ranges 0:1 to 0:255, so they can be blackness values in an image
xout = predictionInstance*255 
# print(xout) # this is diagnostic
# np.shape(xout) # this is diagnostic
flatXOut = xout.flatten()  
# print(len(flatXOut)) # should be 40,000 # this is diagnostic
arr_2d = np.reshape(flatXOut, (200, 200))
# convert float array to ints
array_int = abs(np.array(arr_2d, dtype='int')) 
print(np.shape(array_int)) # this is diagnostic
array_int # this is diagnostic

'''

In [ ]:
"""
draw the image of the output array
"""

''' 
currently doing in func
img = Image.fromarray(array_int.astype(np.uint8))

'''

"""
diagnostic check of the output image

# code:
print("Format: {0}\nSize: {1}\nMode: {2}".format(img.format, 
    img.size, img.mode)) 
# end code
"""


In [ ]:
"""
convert the image file to a recognisable and portable format
write out the image file to somewhere in the local machine
"""

''' 
currently doing in func

pathOut = '/content/drive/MyDrive/BnWOut/batch = 40'
imgRGB = img.convert(mode="RGB")
imgRGB.save(pathOut+"/dreamParamTest1.jpg")
'''

In [34]:
"""
image generation function
can be called solo, compounds all of the post processing into one function, but designed for use in loops
does away with all the diagnostics
"""

def getInstanceImage(pathOut, imgName):
    model = load_model('WARgenerator.h5', compile = False)
    # generate images
    latent_points = generate_latent_points(latent_dim, 1)
    # has quite the think about it
    predictionInstance = model.predict(latent_points)

    # xout = generate_fake_samples(generator, latent_dim, n_samples = 1)[0] # this is being shadowed, I think
    # convert floats from ranges 0:1 to 0:255, so they can be blackness values in an image
    xout = predictionInstance*255 
    flatXOut = xout.flatten()  
    arr_2d = np.reshape(flatXOut, (200, 200))
    # convert float array to ints
    array_int = abs(np.array(arr_2d, dtype='int'))
    img = Image.fromarray(array_int.astype(np.uint8))
    
    # saving image
    imgRGB = img.convert(mode="RGB")
    imgRGB.save(pathOut+"/"+imgName+".jpg")


In [35]:
"""
trains the generator and discriminator parts of the GAN
saves the net model to a .h5 file
"""

def trainAndSave(g_model, d_model, gan_model, dataset, latent_dim, n_epochs, n_batch, filePath, imagePrefix):
    # calculate the number of batches per epoch
    bat_per_epo = int(dataset.shape[0] / n_batch)
    # calculate the total iterations based on batch and epoch
    n_steps = bat_per_epo * n_epochs
    # calculate the number of samples in half a batch
    half_batch = int(n_batch / 2)
    # prepare lists for storing stats each iteration
    d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()
    # manually enumerate epochs
    for i in range(n_steps):
      # get randomly selected 'real' samples
      X_real, y_real = generate_real_samples(dataset, half_batch)
      # update discriminator model weights
      d_loss1, d_acc1 = d_model.train_on_batch(X_real, y_real)
      # generate 'fake' examples
      X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      # update discriminator model weights
      d_loss2, d_acc2 = d_model.train_on_batch(X_fake, y_fake)
      # prepare points in latent space as input for the generator
      X_gan = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = ones((n_batch, 1))
      # update the generator via the discriminator's error
      g_loss = gan_model.train_on_batch(X_gan, y_gan)
      # summarize loss on this batch
      print('>%d/%d, d1=%.3f, d2=%.3f g=%.3f, a1=%d, a2=%d' %
			(i+1, n_steps, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
      # record history
      d1_hist.append(d_loss1)
      d2_hist.append(d_loss2)
      g_hist.append(g_loss)
      a1_hist.append(d_acc1)
      a2_hist.append(d_acc2)
      if (i%10==0): # save frequency, this should really be a smart value
        g_model.save('WARgenerator.h5')
        getInstanceImage(filePath, (imagePrefix+"_e="+str(n_epochs)+"_b="+str(n_batch)+"_lr="+str(learningRate)+"_step_"+str(i)))
    print("finished training")
	  # this is the generator that can be called externally
    g_model.save('WARgenerator.h5')
 
 

In [36]:
trainAndSave(generator, discriminator, gan_model, dataset, latent_dim, n_epochs, n_batch, '/content/drive/MyDrive/BnWOut/breakingItMaybe', 'batchTrain')

>1/440, d1=0.436, d2=0.710 g=0.685, a1=100, a2=0
>2/440, d1=0.259, d2=0.709 g=0.691, a1=100, a2=0
>3/440, d1=0.135, d2=0.699 g=0.706, a1=100, a2=0
>4/440, d1=0.099, d2=0.682 g=0.729, a1=100, a2=100
>5/440, d1=0.061, d2=0.655 g=0.767, a1=100, a2=100
>6/440, d1=0.049, d2=0.614 g=0.817, a1=100, a2=100
>7/440, d1=0.036, d2=0.568 g=0.896, a1=100, a2=100
>8/440, d1=0.047, d2=0.517 g=0.987, a1=100, a2=100
>9/440, d1=0.029, d2=0.454 g=1.084, a1=100, a2=100
>10/440, d1=0.016, d2=0.397 g=1.213, a1=100, a2=100
>11/440, d1=0.030, d2=0.342 g=1.360, a1=100, a2=100
>12/440, d1=0.024, d2=0.290 g=1.536, a1=100, a2=100
>13/440, d1=0.023, d2=0.235 g=1.637, a1=100, a2=100
>14/440, d1=0.016, d2=0.194 g=1.862, a1=100, a2=100
>15/440, d1=0.008, d2=0.163 g=2.011, a1=100, a2=100
>16/440, d1=0.009, d2=0.133 g=2.209, a1=100, a2=100
>17/440, d1=0.017, d2=0.115 g=2.333, a1=100, a2=100
>18/440, d1=0.005, d2=0.092 g=2.512, a1=100, a2=100
>19/440, d1=0.005, d2=0.082 g=2.660, a1=100, a2=100
>20/440, d1=0.009, d2=0.072

KeyboardInterrupt: ignored

In [24]:
learningRate = 0.00005
n_epochs = 30 # number of "generations"
n_batch = 30 # no. of images in each batch

trainAndSave(generator, discriminator, gan_model, dataset, latent_dim, n_epochs, n_batch, '/content/drive/MyDrive/BnWOut/timeSeries7', 'batchTrain')

>1/210, d1=0.686, d2=0.705 g=0.715, a1=80, a2=0
>2/210, d1=0.694, d2=0.708 g=0.716, a1=53, a2=0
>3/210, d1=0.695, d2=0.704 g=0.716, a1=46, a2=0
>4/210, d1=0.692, d2=0.703 g=0.717, a1=60, a2=0
>5/210, d1=0.698, d2=0.700 g=0.718, a1=46, a2=0
>6/210, d1=0.700, d2=0.698 g=0.719, a1=33, a2=0
>7/210, d1=0.706, d2=0.695 g=0.723, a1=20, a2=13
>8/210, d1=0.710, d2=0.693 g=0.722, a1=20, a2=33
>9/210, d1=0.701, d2=0.692 g=0.725, a1=40, a2=100
>10/210, d1=0.709, d2=0.690 g=0.727, a1=6, a2=100
>11/210, d1=0.701, d2=0.691 g=0.725, a1=33, a2=100
>12/210, d1=0.709, d2=0.696 g=0.722, a1=13, a2=0
>13/210, d1=0.698, d2=0.699 g=0.723, a1=40, a2=0
>14/210, d1=0.698, d2=0.699 g=0.733, a1=40, a2=0
>15/210, d1=0.697, d2=0.699 g=0.734, a1=26, a2=0
>16/210, d1=0.696, d2=0.704 g=0.740, a1=33, a2=0
>17/210, d1=0.687, d2=0.704 g=0.743, a1=60, a2=0
>18/210, d1=0.687, d2=0.706 g=0.732, a1=60, a2=0
>19/210, d1=0.671, d2=0.703 g=0.735, a1=86, a2=0
>20/210, d1=0.671, d2=0.711 g=0.712, a1=86, a2=0
>21/210, d1=0.664, d2=

In [26]:
learningRate = 0.00010
n_epochs = 20 # number of "generations"
n_batch = 10 # no. of images in each batch

trainAndSave(generator, discriminator, gan_model, dataset, latent_dim, n_epochs, n_batch, '/content/drive/MyDrive/BnWOut/timeSeries7', 'batchTrain')

>1/440, d1=0.691, d2=0.729 g=0.706, a1=60, a2=0
>2/440, d1=0.679, d2=0.730 g=0.705, a1=60, a2=0
>3/440, d1=0.685, d2=0.733 g=0.703, a1=80, a2=0
>4/440, d1=0.686, d2=0.744 g=0.687, a1=60, a2=0
>5/440, d1=0.670, d2=0.747 g=0.686, a1=80, a2=0
>6/440, d1=0.648, d2=0.745 g=0.694, a1=100, a2=0
>7/440, d1=0.676, d2=0.744 g=0.686, a1=80, a2=0
>8/440, d1=0.664, d2=0.737 g=0.697, a1=100, a2=0
>9/440, d1=0.667, d2=0.731 g=0.702, a1=100, a2=0
>10/440, d1=0.668, d2=0.727 g=0.708, a1=100, a2=0
>11/440, d1=0.667, d2=0.721 g=0.713, a1=100, a2=0
>12/440, d1=0.678, d2=0.720 g=0.713, a1=80, a2=0
>13/440, d1=0.679, d2=0.720 g=0.716, a1=80, a2=0
>14/440, d1=0.665, d2=0.722 g=0.720, a1=100, a2=0
>15/440, d1=0.666, d2=0.729 g=0.704, a1=100, a2=0
>16/440, d1=0.666, d2=0.733 g=0.715, a1=100, a2=0
>17/440, d1=0.681, d2=0.722 g=0.723, a1=100, a2=0
>18/440, d1=0.699, d2=0.721 g=0.707, a1=20, a2=0
>19/440, d1=0.686, d2=0.726 g=0.711, a1=80, a2=0
>20/440, d1=0.679, d2=0.717 g=0.716, a1=100, a2=0
>21/440, d1=0.701, 

In [27]:
learningRate = 0.000001
n_epochs = 20 # number of "generations"
n_batch = 10 # no. of images in each batch

trainAndSave(generator, discriminator, gan_model, dataset, latent_dim, n_epochs, n_batch, '/content/drive/MyDrive/BnWOut/timeSeries7', 'batchTrain')

>1/440, d1=0.672, d2=0.714 g=0.730, a1=60, a2=0
>2/440, d1=0.705, d2=0.733 g=0.701, a1=60, a2=0
>3/440, d1=0.683, d2=0.743 g=0.692, a1=40, a2=0
>4/440, d1=0.667, d2=0.742 g=0.693, a1=100, a2=0
>5/440, d1=0.666, d2=0.742 g=0.700, a1=100, a2=0
>6/440, d1=0.684, d2=0.745 g=0.688, a1=80, a2=0
>7/440, d1=0.652, d2=0.740 g=0.697, a1=100, a2=0
>8/440, d1=0.675, d2=0.723 g=0.716, a1=40, a2=0
>9/440, d1=0.704, d2=0.708 g=0.730, a1=60, a2=0
>10/440, d1=0.703, d2=0.696 g=0.741, a1=40, a2=20
>11/440, d1=0.713, d2=0.688 g=0.747, a1=40, a2=80
>12/440, d1=0.687, d2=0.680 g=0.756, a1=40, a2=100
>13/440, d1=0.684, d2=0.675 g=0.768, a1=60, a2=100
>14/440, d1=0.686, d2=0.664 g=0.781, a1=60, a2=100
>15/440, d1=0.692, d2=0.651 g=0.782, a1=60, a2=100
>16/440, d1=0.701, d2=0.658 g=0.786, a1=40, a2=100
>17/440, d1=0.712, d2=0.665 g=0.782, a1=40, a2=100
>18/440, d1=0.697, d2=0.666 g=0.773, a1=40, a2=100
>19/440, d1=0.687, d2=0.667 g=0.778, a1=60, a2=100
>20/440, d1=0.663, d2=0.668 g=0.773, a1=100, a2=100
>21/4